# User engagement by time, device 

### Pre Analysis assumption and hypothesis        

        i)Relationship between Time and users by year,month, day,hour
        ii)Popular mobile brand and devices
        iii)Most popular mobile brand and device_name        

## Process:

        Explore data
        Clean data
        Plot data 
        Summarize results

# Conducting EDA  data set

In [ ]:
# import pandas and dataset
import pandas as pd
import numpy as np

f = '../data/data_vis_test .csv'
df = pd.read_csv(f)
df.shape

In [ ]:
# look at the observations in each row
df.head(2)

In [ ]:
# check th data type of each object
df.dtypes

### convert time into pandas date time format

In [ ]:
# check and remove null from the time stamp series
len(df[~(df.first_open_timestamp_micros.isnull())]) # to check not null number of rows
df = df[~(df.first_open_timestamp_micros.isnull())]

### time stamp need 19 digits for conversion to standard format(Debugging)

In [ ]:
import datetime as dt     
date = dt.date.today
date()

In [ ]:
pd.to_datetime('today')

In [ ]:
# convertion date and time
pd.to_datetime(1490195805433502912, unit='ns')

In [ ]:
# length of string required for foramt conversion
len(str(1490195805433502912))

In [ ]:
# slice a date time element from the dataframe
first_time_element = df.iloc[1,1]
first_time_element

In [ ]:
#length of time stamp in dataframe
len(str(first_time_element.astype(int)))

In [ ]:
# create a small function to solve this problem
formateed_date_array = np.asarray(list(map(lambda x:x*1000, df.first_open_timestamp_micros.astype(int))))

In [ ]:
#creating a new column
df['time_stamp'] = pd.to_datetime(formateed_date_array, unit = 'ns')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
# recovery copy
df1 = df 
df1.head()

In [ ]:
# creating new columns from time_stamp
df['year'] = df.time_stamp.dt.year 
df['month'] = df.time_stamp.dt.month
df['day'] = df.time_stamp.dt.day
df['hour'] = df.time_stamp.dt.hour

In [ ]:
# first check in place and then remove the columns
df.iloc[:,[0,2,3,5,6,7,8]].head(2)# select columns u want to keep


In [ ]:
# check individual columns year, month, day, hour
print('Unique years \n',df.time_stamp.dt.year.value_counts())
print('\nUnique month\n\n',df.time_stamp.dt.month.value_counts())
print('\nUnique day\n',df.time_stamp.dt.day.value_counts())
print('\nWeekdays name\n\n',df.time_stamp.dt.weekday_name.value_counts())
print('\nUnique hour\n\n',df.time_stamp.dt.hour.value_counts())

## Data Cleaning Missing value 

In [ ]:
#checking for Null in the data
print("User_id nulls:", len(df[df.user_id.isnull()]))
print("Mobile nulls:", len(df[df.mobile_brand_name.isnull()]))
print("Device nulls:", len(df[df.device_model.isnull()]))
print("Time stamp nulls:", len((df[df.time_stamp.isnull()]) ))   

In [ ]:
print('Error % due to missing mobile data' ,617/10000)

In [ ]:
# Removing records with no user_id
df = df[df.user_id.notnull()]

In [ ]:
df.columns

### We using ML to fill missing values

In [ ]:
# Creting a back copy 
clean_df = df

### MAPPING categorical value to numerical for better accuracy

In [ ]:
mobile_brand_name_list = list(set(clean_df.mobile_brand_name.values))
device_model_list = list(set(clean_df.device_model.values))

In [ ]:
len(mobile_brand_name_list)

In [ ]:
mobile_name_for_mapping = [{brand_name:index} for index,brand_name in enumerate(mobile_brand_name_list)]

In [ ]:
mobile_name_dict_for_mapping = {}
for index,brand_name in enumerate(mobile_brand_name_list):
    mobile_name_dict_for_mapping[brand_name] = index 

In [ ]:
clean_df.loc[:,'mapped_mobile_brand_name'] = clean_df.mobile_brand_name.map(mobile_name_dict_for_mapping)

In [ ]:
len(device_model_list)

In [ ]:
# we start this series with 1000 to differenciate this
device_model_for_mapping = [{device_name:1000 + index} for index,device_name in enumerate(device_model_list)]

In [ ]:
#creating a dictionary for mapping

device_model_dict_for_mapping = {}
for index,device_model in enumerate(device_model_list):
    device_model_dict_for_mapping[device_model] = index 

In [ ]:
#mapping and encoding the device column as numbers 
clean_df.loc[:,'mapped_device_model'] = clean_df.device_model.map(device_model_dict_for_mapping)

In [ ]:
clean_df.head(2)

#### so for all mobile brand name = NaN we have 0

In [ ]:
#slice with nan elements from table

clean_df[clean_df.loc[:,'mapped_mobile_brand_name'] == 0]

#### creating a validation set  to test model accuracy

In [ ]:
transformed_df = clean_df.loc[:,['mapped_device_model']].apply(lambda x: x.astype('category'))
transformed_df['mapped_mobile_brand_name'] = clean_df.loc[:,['mapped_mobile_brand_name']].apply(lambda x: x.astype('category'))

### SPLITING TEST FROM TRAINING SET

In [ ]:
train_X = transformed_df[transformed_df.mapped_mobile_brand_name != 0 ].mapped_device_model 
train_y = transformed_df[transformed_df.mapped_mobile_brand_name != 0].mapped_mobile_brand_name
test_X = transformed_df[transformed_df.mapped_mobile_brand_name == 0].mapped_device_model
test_y = transformed_df[transformed_df.mapped_mobile_brand_name == 0].mapped_mobile_brand_name# MISSING VALUES WE NEED TO FIND

In [ ]:
test_y.shape

### next : creating a validation set for our model

In [ ]:
from sklearn.cross_validation import train_test_split

X1_train,X1_test,y1_train,y1_test = train_test_split(train_X, train_y,random_state = 0)

In [ ]:
X1_train.values.astype(int)

### IMprovising the accuracy of the model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#RandomForestClassifier?
forest2 = RandomForestClassifier(n_estimators= 72, max_depth = 23)
forest2.fit(np.asarray(X1_train).reshape(-1,1),np.asarray(y1_train))
m1 = forest2.predict(np.asarray(X1_test).reshape(-1,1))
m2 = np.asarray(y1_test).reshape(-1,1)

In [ ]:
from sklearn import metrics
metrics.accuracy_score(m1.reshape(-1,1),m2)

In [ ]:
from sklearn.svm import SVC # "Support Vector Classifier"
clf = SVC()
clf.fit(np.asarray(X1_train).reshape(-1,1),np.asarray(y1_train) )
#SVC
m9 = clf.predict(np.asarray(X1_test).reshape(-1,1))
n9 = np.asarray(y1_test).reshape(-1,1)

In [ ]:
from sklearn import metrics
metrics.accuracy_score(m9.reshape(-1,1),n9)

### Model accuracy freezes at 96.16%


Selecting random forest model hyperparameters (n_estimators= 72, max_depth = 23)

Fitting model on orginal training set(mergeing validation set with it to fit the model)
train_X 
train_y

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#RandomForestClassifier?
forest3 = RandomForestClassifier(n_estimators= 72, max_depth = 23)
forest3.fit(np.asarray(train_X).reshape(-1,1),np.asarray(train_y))
predicted_y = forest3.predict(np.asarray(test_X).reshape(-1,1))


In [ ]:
# how to reverse key value in a 
#mobile_name_dict_for_mapping
inv_mobile_name_dict_for_mapping = {value: key for key, value in mobile_name_dict_for_mapping.items()}

In [ ]:
# for those already there
transformed_df['cleaned_mobile_name_correct']= transformed_df[transformed_df.mapped_mobile_brand_name != 0].mapped_mobile_brand_name.map(inv_mobile_name_dict_for_mapping )

In [ ]:
transformed_df.head(5)

### Series of steps to decode encoded mobile_name 

In [ ]:
correct_row = transformed_df[transformed_df.mapped_mobile_brand_name != 0].mapped_mobile_brand_name

In [ ]:
a = transformed_df[transformed_df.mapped_mobile_brand_name == 0].mapped_mobile_brand_name

In [ ]:
transformed_df.columns

In [ ]:
c = transformed_df[transformed_df.mapped_mobile_brand_name != 0].mapped_mobile_brand_name

In [ ]:
correct_row_index = c.index

In [ ]:
missing_row_index = a.index

In [ ]:
correct_row = transformed_df[transformed_df.mapped_mobile_brand_name != 0].mapped_mobile_brand_name

In [ ]:
missing_row = pd.Series(predicted_y, index = missing_row_index)

In [ ]:
cleaned_mobile_brand = correct_row.append(missing_row).sort_index()

In [ ]:
transformed_df['cleaned_mobile_name_encoded'] = cleaned_mobile_brand

In [ ]:
transformed_df.head(2)

In [ ]:
#cleaned mobile name is the name of column with all correct names

transformed_df['cleaned_mobile_name'] = transformed_df['cleaned_mobile_name_encoded'].map(inv_mobile_name_dict_for_mapping)

### Check the tranformed for any error (6th row NaN predicted as Samsung )

In [ ]:
transformed_df.head(10)

In [ ]:
clean_df.head(1)

In [ ]:
pd.concat([clean_df,transformed_df['cleaned_mobile_name']],axis = 1).head(5)

In [ ]:
clean_df = pd.concat([clean_df,transformed_df['cleaned_mobile_name']],axis = 1)

In [ ]:
clean_df.head(2)

In [ ]:
#selecting column names to delete
exclude_cols = ['first_open_timestamp_micros','mobile_brand_name','mapped_mobile_brand_name','mapped_device_model']
#export clean

In [ ]:
# deleting columns we donot need to reduce clutter
clean_df.drop(exclude_cols, axis=1, inplace=True)

In [ ]:
clean_df = clean_df.rename(index=str, columns={'cleaned_mobile_name': "mobile_brand_name"})
clean_df.iloc[:,[0,7,1,2,3,4,5,6]].head(2)

In [ ]:
clean_df =  clean_df.iloc[:,[0,7,1,2,3,4,5,6]]

In [ ]:
cleaned_cols = clean_df.columns 

In [ ]:
clean_df.head(2)

In [ ]:
clean_df.shape

## Exporting cleaned data set

In [ ]:
clean_df.to_csv('clean_df.csv',columns=cleaned_cols, header=True,compression=None, encoding='UTF-8')

## Creating Tidy Datasets


In [ ]:
tidy_df = clean_df
tidy_df.columns

In [ ]:
# tidy_df will we used for grouping and further analysis
tidy_df.time_stamp.dt.weekday_name.value_counts()

In [ ]:
# tidy user by weekdays
user_by_brands = tidy_df.iloc[:,[0,7]]
user_by_brands.head(2)

In [ ]:
tidy_df.dtypes

## Group by all variables

In [ ]:
# groupping user id by all parameters
all_groupi = tidy_df.groupby(['year','month','day','hour'])

In [ ]:
td = tidy_df.groupby(['year','month','day','hour'])['user_id'].count()

In [ ]:
pd.DataFrame(td)

In [ ]:
pd.DataFrame(td).info()

### group_by mobile brand name

In [ ]:
group_by_brand = df.groupby('mobile_brand_name') 

In [ ]:
group_by_brand.count()

### group_by mobile brand and device

In [ ]:
group_by_model_sub_model = df.groupby(['mobile_brand_name','device_model'])

In [ ]:
group_by_model_sub_model.count().user_id